In [2]:
import numpy as np
np.random.seed(123)
import sys
sys.path.append('..')


from util import *

%load_ext autoreload
%autoreload 2

In [3]:
def P_M(u, m):
    """
    u: current iterate
    m: true magnitude
    """
    F_u = np.fft.fft2(u)
    F_u += (F_u == 0) # Multiply m by one for zero Fourier entries
    v0_hat = m * (F_u / np.abs(F_u))
    return np.real(np.fft.ifft2(v0_hat))

def P_Spos(u, m, D=None):
    """
    u: current iterate
    m: true magnitude
    D: support mask, None = full support
    """
    if D is None:
        return np.maximum(0, u)
    else:
        return np.maximum(0, u) * D
    
def RAAR(m, beta=0.8, steps=200, verbose=True, D=None):

    u = np.random.rand(*m.shape)
    
    R_M = lambda u: 2 * P_M(u, m) - u
    R_Spos = lambda u: 2 * P_Spos(u, m) - u  
    
    for s in range(steps):
        T = 0.5 * (R_Spos(R_M(u)) + u)
        u = beta * T + (1 - beta) * P_M(u, m)
    
    return np.real(u)

In [4]:
import sys
sys.path.append('..')

In [5]:
def poisson_noise(magn, alpha=1):
    intens = magn ** 2
    alpha_2 = alpha ** 2
    lmd = intens / alpha_2
    intens_noise = alpha_2 * np.random.poisson(lmd, magn.shape)
    magn_noise = np.sqrt(intens_noise)
    return magn_noise, intens_noise

In [6]:
from data import load
test_data = load(name='MNIST', path='../data/mnist/', batch_size=1024)[0]['test']
batch = next(iter(test_data)).numpy()

In [7]:
alphas = [0.01, 0.03, 0.1, 0.3, 1.0, 3.0, 10.0, 25.0, 50.0]

for alpha in alphas:
    results = np.zeros_like(batch)
    for i, image in enumerate(batch):
        if i % 10==0:
            print(i)
        magnitudes, _ = poisson_noise(np.abs(np.fft.fft2(image)), alpha=alpha)
        res = []
        errors = []
        for _ in range(3):
            res.append(RAAR(magnitudes, steps=1000, verbose=False))
            errors.append(np.mean((np.abs(np.fft.fft2(res[-1]))-magnitudes)**2))     
        results[i] = res[np.argmin(errors)]
        
        import pickle
    pickle.dump(batch, open("RAAR_MNIST/original_alpha{}.p".format(alpha), "wb"))
    pickle.dump(results, open("RAAR_MNIST/predicted_noise_alpha{}.p".format(alpha), "wb"))

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500

In [8]:
### CACHE RESULTS ###

import pickle
pickle.dump(batch, open("RAAR_MNIST/original_alpha{}.p".format(alpha), "wb"))
pickle.dump(results, open("RAAR_MNIST/predicted_noise_alpha{}.p".format(alpha), "wb"))

In [37]:
### LOAD RESULTS ###
import pickle

alpha = alphas[8]

batch = pickle.load(open("RAAR_MNIST/original_alpha{}.p".format(alpha), "rb"))
results = pickle.load(open("RAAR_MNIST/predicted_noise_alpha{}.p".format(alpha), "rb"))

In [38]:
from util import benchmark
benchmark(results, batch, check_all=True)
print(alpha)

Signal error:
  MSE: 0.1339909883135988, std: 0.04746560847341171
  MAE: 0.2932611624637502, std: 0.05837631375211292
  SSIM: 0.018144266549690834, std: 0.05478680190194163
  Sharpness: 0.22219386159234666, std: 0.04934185697146765
  PhCo-MSE: 0.12360992316280317, std: 0.046584425243217195
  PhCo-MAE: 0.2823277310890262, std: 0.05678730837392257
  PhCo-SSIM: 0.0755344248204842, std: 0.0746831257931448
  CroCo-MSE: 0.09035240124285338, std: 0.02554114299413436
  CroCo-MAE: 0.246620221354533, std: 0.04189295345623957
  CroCo-SSIM: 0.20365295665992966, std: 0.06984610290229992

Magnitude error:
  MSE Magnitude: 43.86296586599201, std: 11.209581298749539

Image constraints:
  Imag part = 0.0 - should be very close to 0
  Real part is in [0.00, 2.12] - should be in [0, 1]

Markdown table values:
 0.1340 | 0.2933 | 0.01814 | 0.2222 | 0.1236 | 0.2823 | 0.07553 | 0.09035 | 0.2466 | 0.2037 | 43.86 |
50.0


## RAAR MNIST


| alpha | MSE | MAE | SSIM | SHRP | PhaCo-MSE | PhaCo-MAE | PhaCo-SSIM | CroCo-MSE | CroCo-MAE | CroCo-SSIM | MSE Magn |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
0.01|0.1696 | 0.2232 | 0.08557 | 0.1585 | 0.06322 | 0.1229 | 0.4731 | 0.04988 | 0.1101 | 0.5271 | 1.142 |
0.03| 0.1695 | 0.2231 | 0.08667 | 0.1569 | 0.06184 | 0.1208 | 0.4827 | 0.04944 | 0.1091 | 0.5321 | 1.116 |
0.1| 0.1700 | 0.2236 | 0.08732 | 0.1563 | 0.06202 | 0.1209 | 0.4819 | 0.04875 | 0.1083 | 0.5349 | 1.112 |
0.3| 0.1701 | 0.2244 | 0.08285 | 0.1583 | 0.06198 | 0.1219 | 0.4729 | 0.04936 | 0.1099 | 0.5241 | 1.154 |
1.0| 0.1687 | 0.2275 | 0.05482 | 0.1666 | 0.06449 | 0.1296 | 0.4179 | 0.05099 | 0.1169 | 0.4689 | 1.542 |
3.0| 0.1649 | 0.2378 | 0.03165 | 0.1815 | 0.06466 | 0.1444 | 0.3693 | 0.05170 | 0.1320 | 0.4186 | 3.494 |
10.0| 0.1580 | 0.2650 | 0.01861 | 0.2084 | 0.07856 | 0.1902 | 0.2888 | 0.06031 | 0.1727 | 0.3547 | 12.31 |
25.0 |0.1475 | 0.2898 | 0.01595 | 0.2222 | 0.1131 | 0.2555 | 0.1467 | 0.07586 | 0.2177 | 0.2817 | 28.54 |
50.0 | 0.1340 | 0.2933 | 0.01814 | 0.2222 | 0.1236 | 0.2823 | 0.07553 | 0.09035 | 0.2466 | 0.2037 | 43.86 |

In [60]:
raar_croco = [0.04875, 0.04936, 0.05099, 0.05170, 0.06031, 0.07586, 0.09035]

In [11]:
test_data = load(name='Fashion', path='../data/fashion/', batch_size=1024)[0]['test']
batch = next(iter(test_data)).numpy()

  1%|          | 212992/26421880 [00:00<00:13, 2013517.00it/s]

Fashion


26427392it [00:00, 59522622.66it/s]                             


Extracting ../data/fashion/FashionMNIST/raw/train-images-idx3-ubyte.gz to ../data/fashion/FashionMNIST/raw


32768it [00:00, 640687.28it/s]
  2%|▏         | 98304/4422102 [00:00<00:04, 932693.75it/s]

Extracting ../data/fashion/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ../data/fashion/FashionMNIST/raw


4423680it [00:00, 18009070.14it/s]                            
8192it [00:00, 267711.80it/s]

Extracting ../data/fashion/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ../data/fashion/FashionMNIST/raw
Extracting ../data/fashion/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/fashion/FashionMNIST/raw
Processing...
Done!


In [12]:
alphas = [0.01, 0.03, 0.1, 0.3, 1.0, 3.0, 10.0, 25.0, 50.0]

for alpha in alphas:
    results = np.zeros_like(batch)
    for i, image in enumerate(batch):
        if i % 100==0:
            print(i)
        magnitudes, _ = poisson_noise(np.abs(np.fft.fft2(image)), alpha=alpha)
        res = []
        errors = []
        for _ in range(3):
            res.append(RAAR(magnitudes, steps=1000, verbose=False))
            errors.append(np.mean((np.abs(np.fft.fft2(res[-1]))-magnitudes)**2))     
        results[i] = res[np.argmin(errors)]
        
        import pickle
    pickle.dump(batch, open("RAAR_Fashion/original_alpha{}.p".format(alpha), "wb"))
    pickle.dump(results, open("RAAR_Fashion/predicted_noise_alpha{}.p".format(alpha), "wb"))

0
100
200
300
400
500
600
700
800
900
1000
0
100
200
300
400
500
600
700
800
900
1000
0
100
200
300
400
500
600
700
800
900
1000
0
100
200
300
400
500
600
700
800
900
1000
0
100
200
300
400
500
600
700
800
900
1000
0
100
200
300
400
500
600
700
800
900
1000
0
100
200
300
400
500
600
700
800
900
1000
0
100
200
300
400
500
600
700
800
900
1000
0
100
200
300
400
500
600
700
800
900
1000


In [13]:
### CACHE RESULTS ###

import pickle
pickle.dump(batch, open("RAAR_Fashion/original_alpha{}.p".format(alpha), "wb"))
pickle.dump(results, open("RAAR_Fashion/predicted_noise_alpha{}.p".format(alpha), "wb"))

In [58]:
### LOAD RESULTS ###
import pickle

alpha = alphas[8]

batch = pickle.load(open("RAAR_Fashion/original_alpha{}.p".format(alpha), "rb"))
results_fashion = pickle.load(open("RAAR_Fashion/predicted_noise_alpha{}.p".format(alpha), "rb"))

In [59]:
benchmark(results_fashion, batch, check_all=True)#
print(alpha)

Signal error:
  MSE: 0.18318988758346677, std: 0.1116078321040198
  MAE: 0.34101027300494025, std: 0.11892892545088828
  SSIM: 0.03832860102512371, std: 0.10871922635105967
  Sharpness: 0.22572929582436663, std: 0.05375008216903813
  PhCo-MSE: 0.1963843710072979, std: 0.11565863318583586
  PhCo-MAE: 0.35711195357725956, std: 0.12172467304405764
  PhCo-SSIM: 0.04217467702005959, std: 0.10140761337098635
  CroCo-MSE: 0.07428774456730025, std: 0.03476693016525124
  CroCo-MAE: 0.2151633988833055, std: 0.051454958434805034
  CroCo-SSIM: 0.2916552565214826, std: 0.08903180187391681

Magnitude error:
  MSE Magnitude: 29.54925695899874, std: 10.650881622984793

Image constraints:
  Imag part = 0.0 - should be very close to 0
  Real part is in [0.00, 2.10] - should be in [0, 1]

Markdown table values:
 0.1832 | 0.3410 | 0.03833 | 0.2257 | 0.1964 | 0.3571 | 0.04217 | 0.07429 | 0.2152 | 0.2917 | 29.55 |
50.0


## RAAR Fashion


| alpha | MSE | MAE | SSIM | SHRP | PhaCo-MSE | PhaCo-MAE | PhaCo-SSIM | CroCo-MSE | CroCo-MAE | CroCo-SSIM | MSE Magn |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | 
0.01| 0.2159 | 0.3349 | 0.06492 | 0.2183 | 0.1209 | 0.2259 | 0.3209 | 0.06685 | 0.1665 | 0.4323 | 0.01654 |
0.03|  0.2219 | 0.3415 | 0.06066 | 0.2180 | 0.1190 | 0.2240 | 0.3169 | 0.06668 | 0.1664 | 0.4314 | 0.01581 |
0.1|0.2223 | 0.3409 | 0.06172 | 0.2181 | 0.1211 | 0.2260 | 0.3175 | 0.06640 | 0.1658 | 0.4331 | 0.01686 |
0.3|0.2208 | 0.3392 | 0.06205 | 0.2191 | 0.1224 | 0.2285 | 0.3087 | 0.06707 | 0.1671 | 0.4286 | 0.03006 |
1.0| 0.2158 | 0.3363 | 0.05477 | 0.2205 | 0.1248 | 0.2319 | 0.2886 | 0.06699 | 0.1688 | 0.4044 | 0.2035 |
3.0|0.2112 | 0.3366 | 0.04632 | 0.2230 | 0.1267 | 0.2411 | 0.2524 | 0.06659 | 0.1744 | 0.3726 | 1.544 |
10.0 |0.2035 | 0.3400 | 0.03857 | 0.2237 | 0.1639 | 0.2988 | 0.1467 | 0.06653 | 0.1869 | 0.3481 | 7.873 |
25.0 | 0.1941 | 0.3437 | 0.03407 | 0.2241 | 0.2028 | 0.3560 | 0.04655 | 0.06862 | 0.2005 | 0.3252 | 18.05 |
50.0|0.1832 | 0.3410 | 0.03833 | 0.2257 | 0.1964 | 0.3571 | 0.04217 | 0.07429 | 0.2152 | 0.2917 | 29.55 |

In [ ]:
raar_croco = [0.06640, 0.06707, 0.06699, 0.06659, 0.06653, 0.06862 ,0.07429]